In [7]:
import os
os.chdir(r"c:\Users\britt\Desktop\YH\Applicerad AI\job_discrimination_sandbox")
import warnings

import gensim
import numpy as np
import optuna
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [8]:
# warnings.simplefilter("ignore")

In [9]:
google_model = gensim.models.KeyedVectors.load_word2vec_format("c:/Users/britt/Downloads/GoogleNews-vectors-negative300.bin.gz", binary=True)

In [10]:
df = pd.read_csv("data/cleaned_data/bulletins_labels_share_content.csv", dtype={'ID': object})
df

,ID,Job Description,Apps Received,Female,Male,Unknown_Gender,Cleaned text,Apps Received (unknown gender removed),Male share,Female share,Male share (unknown gender included),Female share (unknown gender included)
0,9206,311 DIRECTOR,54,20,31,3,director class code open date annual salary du...,51,0.608,0.392,0.574,0.370
1,1223,ACCOUNTING CLERK,648,488,152,8,accounting clerk class code open date exam ope...,640,0.238,0.762,0.235,0.753
2,7260,AIRPORT MANAGER,51,13,37,1,airport manager class code open date exam open...,50,0.740,0.260,0.725,0.255
3,3227,AIRPORT POLICE LIEUTENANT,48,9,38,1,airport police lieutenant class code open date...,47,0.809,0.191,0.792,0.188
4,2400,AQUARIST,40,15,24,1,aquarist class code open date annual salary ca...,39,0.615,0.385,0.600,0.375
...,...,...,...,...,...,...,...,...,...,...,...,...
172,7840,WASTEWATER TREATMENT LABORATORY MANAGER,16,6,9,1,wastewater treatment laboratory manager class ...,15,0.600,0.400,0.562,0.375
173,4123,WASTEWATER TREATMENT OPERATOR,125,9,113,3,wastewater treatment operator class code open ...,122,0.926,0.074,0.904,0.072
174,7857,WATER MICROBIOLOGIST,179,89,82,8,water microbiologist class code open date revi...,171,0.480,0.520,0.458,0.497
175,3912,WATER UTILITY WORKER,96,2,92,2,water utility worker class code open date exam...,94,0.979,0.021,0.958,0.021


In [11]:
corpus = list(df["Cleaned text"])
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit_transform(corpus)

<177x3835 sparse matrix of type '<class 'numpy.float64'>'
	with 68003 stored elements in Compressed Sparse Row format>

In [12]:
vocabulary = tfidf_vectorizer.get_feature_names_out()
documents_embeddings = []
documents_scaled_embeddings = []
for doc in corpus:
    word_embeddings = []
    scaled_embeddings  = []
    doc_list = doc.split()
    for word in doc_list:
        if word in google_model.key_to_index.keys():
            embedding = google_model[word]
            word_embeddings.append(embedding)
            index = np.where(vocabulary == word)[0]
            try:
                scaled_embeddings.append(embedding * tfidf_vectorizer.idf_[index])
            except ValueError:
                pass
    documents_embeddings.append(word_embeddings)
    documents_scaled_embeddings.append(scaled_embeddings)

In [14]:
vocabulary

array(['ab', 'abatement', 'abbreviation', ..., 'youth', 'zero', 'zone'],
      dtype=object)

In [13]:
len(documents_embeddings)

177

In [69]:
doc_vectors = [np.average(doc, axis=0) for doc in documents_embeddings]
X_emb = np.array(doc_vectors)
print(X_emb.shape)
X_emb[0]

(177, 300)


array([-0.04414452, -0.0122033 ,  0.00511145,  0.04091902, -0.03405884,
       -0.01762165,  0.08182327, -0.05525649,  0.06198912, -0.0112897 ,
       -0.03507755, -0.04316111, -0.015784  ,  0.02444414, -0.07486102,
        0.11535516,  0.04934856,  0.03701751, -0.02934492, -0.06692612,
       -0.00137953, -0.01058333, -0.04252194,  0.04651456,  0.00205325,
       -0.05423673, -0.10081883,  0.04333396, -0.02217435,  0.00980085,
        0.01191099, -0.05856771, -0.05402977, -0.01316072,  0.01299298,
       -0.03552402,  0.03439947,  0.01501433,  0.04625956, -0.01263558,
        0.04732609, -0.0170915 ,  0.04642782,  0.01835999, -0.09528378,
       -0.11837751, -0.04706389,  0.00509897, -0.10190953,  0.04831512,
        0.003875  , -0.0154472 , -0.0525268 , -0.00877736, -0.02762537,
        0.03335165, -0.04236813, -0.06015126,  0.0071311 , -0.0569754 ,
       -0.04933222,  0.00871245, -0.11213428, -0.0544597 , -0.00634763,
        0.01197628, -0.06366016,  0.08447613,  0.02362785,  0.02

In [70]:
scaled_doc_vectors = [np.average(doc, axis=0) for doc in documents_scaled_embeddings]
X_scaled = np.array(scaled_doc_vectors)
print(X_scaled.shape)
X_scaled[0]

(177, 300)


array([-0.05982076, -0.01256929,  0.00738425,  0.06441563, -0.06178074,
       -0.02890935,  0.13246215, -0.09674023,  0.09870001, -0.01482921,
       -0.05794048, -0.08452343, -0.02373681,  0.02202474, -0.11803477,
        0.16630183,  0.06023616,  0.05480083, -0.05190434, -0.09978531,
        0.00942413,  0.00640978, -0.05457966,  0.07560081, -0.00125979,
       -0.0792425 , -0.14896783,  0.07502251, -0.0335416 , -0.0064042 ,
        0.01995128, -0.08036403, -0.10809168, -0.0314087 ,  0.03789691,
       -0.08254778,  0.04851855,  0.02704024,  0.08806353, -0.01758198,
        0.08302844, -0.03782095,  0.06620392,  0.02680038, -0.16658106,
       -0.20945015, -0.09592282,  0.00920302, -0.14081313,  0.09008291,
        0.00177784, -0.01586967, -0.08783568, -0.01162592, -0.04698426,
        0.03645247, -0.06814428, -0.11285288,  0.00941496, -0.09028452,
       -0.09941413,  0.01527861, -0.1703441 , -0.07954433, -0.0044887 ,
        0.0195754 , -0.110327  ,  0.15459884,  0.04444556,  0.03

In [118]:
X_tfidf = tfidf_vectorizer.fit_transform(corpus).toarray()

In [116]:
y_classes = list(df["Numeric label 70/30"])
y_male_share = list(df["Male share"])
y_multi_share = list(zip(list(df["Male share (unknown gender included)"]), list(df["Female share (unknown gender included)"])))

In [117]:
y_multi_share

[(0.574, 0.37),
 (0.235, 0.753),
 (0.725, 0.255),
 (0.792, 0.188),
 (0.6, 0.375),
 (0.41, 0.553),
 (0.471, 0.52),
 (0.342, 0.613),
 (0.962, 0.029),
 (0.458, 0.521),
 (0.894, 0.082),
 (0.721, 0.26),
 (0.968, 0.016),
 (0.96, 0.035),
 (0.987, 0.0),
 (0.889, 0.111),
 (0.103, 0.897),
 (0.596, 0.386),
 (0.378, 0.545),
 (0.4, 0.6),
 (0.846, 0.154),
 (0.929, 0.071),
 (0.982, 0.005),
 (0.972, 0.028),
 (0.88, 0.065),
 (0.282, 0.705),
 (0.318, 0.677),
 (0.637, 0.357),
 (0.643, 0.357),
 (0.929, 0.071),
 (0.8, 0.2),
 (0.987, 0.0),
 (0.971, 0.029),
 (0.95, 0.031),
 (1.0, 0.0),
 (0.965, 0.009),
 (0.995, 0.005),
 (0.639, 0.329),
 (0.953, 0.016),
 (0.818, 0.152),
 (0.78, 0.198),
 (0.686, 0.296),
 (0.957, 0.032),
 (0.95, 0.01),
 (0.891, 0.096),
 (0.514, 0.448),
 (0.615, 0.369),
 (0.832, 0.15),
 (0.952, 0.036),
 (0.649, 0.339),
 (0.595, 0.381),
 (0.979, 0.018),
 (0.849, 0.144),
 (0.445, 0.545),
 (0.429, 0.571),
 (0.703, 0.243),
 (0.514, 0.462),
 (0.718, 0.239),
 (0.539, 0.435),
 (0.4, 0.6),
 (0.734, 0.25

In [49]:
skf = StratifiedKFold(n_splits=10)

# for train_index, test_index in skf.split(X_emb, y_classes):
#     print("TRAIN:", train_index)
#     print("TEST:", test_index)
#     X_train, X_test = X_emb[train_index], X_emb[test_index]
#     y_train, y_test = y_classes[train_index], y_classes[test_index]

folds_dicts = []
for i, (train_index, test_index) in enumerate(skf.split(X_emb, y_classes)):
        print(f"Fold {i}:")
        print(f"  Train: index={train_index}")
        print(f"  Test:  index={test_index}")
        folds_dict = {"Train indices": train_index,
                      "Test indices": test_index}
        folds_dicts.append(folds_dict)


Fold 0:
  Train: index=[ 16  18  19  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176]
  Test:  index=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 17 20]
Fold 1:
  Train: index=[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  17  20
  25  35  36  38  39  40  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  

In [33]:
y_classes[19]

0

In [123]:
folds_content_dicts = []
for i, fold_dict in enumerate(folds_dicts):
    train_y_classes = []
    train_y_male_share = []
    train_y_multi_share = []
    train_X_tfidf = []
    train_X_emb = []
    train_X_scaled = []
    for train_index in fold_dict["Train indices"]:
        train_y_classes.append(y_classes[train_index])
        train_y_male_share.append(y_male_share[train_index])
        train_y_multi_share.append(y_multi_share[train_index])
        train_X_tfidf.append(X_tfidf[train_index])
        train_X_emb.append(X_emb[train_index])
        train_X_scaled.append(X_scaled[train_index])
    test_y_classes = []
    test_y_male_share = []
    test_y_multi_share = []
    test_X_tfidf = []
    test_X_emb = []
    test_X_scaled = []
    for test_index in fold_dict["Test indices"]:
        test_y_classes.append(y_classes[test_index])
        test_y_male_share.append(y_male_share[test_index])
        test_y_multi_share.append(y_multi_share[test_index])
        test_X_tfidf.append(X_tfidf[test_index])
        test_X_emb.append(X_emb[test_index])
        test_X_scaled.append(X_scaled[test_index])
    fold_labels_dict = {
        "Train y classes": train_y_classes,
        "Train y male share": train_y_male_share,
        "Train y multi share": train_y_multi_share,
        "Train X Tfidf": train_X_tfidf,
        "Train X embeddings": train_X_emb,
        "Train X scaled embeddings": train_X_scaled,
        "Test y classes": test_y_classes,
        "Test y male share": test_y_male_share,
        "Test y multi share": test_y_multi_share,
        "Test X Tfidf": test_X_tfidf,
        "Test X embeddings": test_X_emb,
        "Test X scaled embeddings": test_X_scaled
        }
    folds_content_dicts.append(fold_labels_dict)


In [124]:
for i, fold_labels_dict in enumerate(folds_content_dicts):
    train_labels = fold_labels_dict["Train y classes"]
    test_labels = fold_labels_dict["Test y classes"]
    print(f"Fold {i}:")
    print(f"Number of train labels: {len(train_labels)}")
    print(f"Train labels: {train_labels}")
    print(f"Occurrences label N: {train_labels.count(0)}")
    print(f"Occurrences label F: {train_labels.count(1)}")
    print(f"Occurrences label M: {train_labels.count(2)}")
    print()
    print(f"Number of test labels: {len(test_labels)}")
    print(f"Test labels: {test_labels}")
    print(f"Occurrences label N: {test_labels.count(0)}")
    print(f"Occurrences label F: {test_labels.count(1)}")
    print(f"Occurrences label M: {test_labels.count(2)}")
    print()

Fold 0:
Number of train labels: 159
Train labels: [1, 0, 0, 2, 2, 2, 2, 1, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 1, 2, 2, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 2, 0, 2, 2, 0, 0, 1, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 1, 2, 0, 2, 2, 2, 0, 2, 2, 0, 0, 0, 2, 0, 2, 0]
Occurrences label N: 55
Occurrences label F: 9
Occurrences label M: 95

Number of test labels: 18
Test labels: [0, 1, 2, 2, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2]
Occurrences label N: 7
Occurrences label F: 1
Occurrences label M: 10

Fold 1:
Number of train labels: 159
Train labels: [0, 1, 2, 2, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 1, 2, 2, 2, 0, 0, 1, 2, 0, 0, 0

In [45]:
list(df["Label 70/30"]).count("W")

10

In [46]:
list(df["Label 70/30"]).count("M")

105

In [47]:
list(df["Label 70/30"]).count("N")

62

In [64]:
clf = LogisticRegression(C=8.665886068370195, class_weight="balanced")
#clf.fit(X_emb, y_classes)

In [71]:
scores = cross_val_score(clf, X_emb, y_classes, cv=10)
scores

array([0.88888889, 0.72222222, 0.88888889, 0.44444444, 0.83333333,
       0.88888889, 0.83333333, 0.82352941, 0.70588235, 0.82352941])

In [72]:
scores.mean()

0.7852941176470589

In [73]:
scores = cross_val_score(clf, X_scaled, y_classes, cv=10)
scores

array([0.88888889, 0.72222222, 0.94444444, 0.55555556, 0.94444444,
       0.94444444, 0.88888889, 0.94117647, 0.64705882, 0.94117647])

In [74]:
scores.mean()

0.8418300653594771

In [125]:
def objective(trial):
    C = trial.suggest_float("C", 1e-7, 10.0, log=True)
    c_weight = trial.suggest_categorical("c_weight", ["balanced", None])

    clf = LogisticRegression(C=C, class_weight=c_weight)
    scores = []
    for fold_labels_dict in folds_content_dicts[:-1]:
        clf.fit(fold_labels_dict["Train X scaled embeddings"], fold_labels_dict["Train y classes"])
        scores.append(clf.score(fold_labels_dict["Test X scaled embeddings"], fold_labels_dict["Test y classes"]))

    return np.mean(scores)

In [126]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=500)

[I 2023-01-24 09:49:37,979] A new study created in memory with name: no-name-5b79c75d-8d68-41db-bb1e-44c6e25cb76b
[I 2023-01-24 09:49:38,123] Trial 0 finished with value: 0.593681917211329 and parameters: {'C': 0.01620309706284892, 'c_weight': None}. Best is trial 0 with value: 0.593681917211329.
[I 2023-01-24 09:49:38,319] Trial 1 finished with value: 0.593681917211329 and parameters: {'C': 1.1819692753219637e-06, 'c_weight': None}. Best is trial 0 with value: 0.593681917211329.
[I 2023-01-24 09:49:38,361] Trial 2 finished with value: 0.6372549019607843 and parameters: {'C': 8.461166915411248e-05, 'c_weight': 'balanced'}. Best is trial 2 with value: 0.6372549019607843.
[I 2023-01-24 09:49:38,440] Trial 3 finished with value: 0.593681917211329 and parameters: {'C': 0.0038478945299672152, 'c_weight': None}. Best is trial 2 with value: 0.6372549019607843.
[I 2023-01-24 09:49:38,552] Trial 4 finished with value: 0.593681917211329 and parameters: {'C': 2.2633198672785516e-06, 'c_weight': N

In [127]:
study.best_params

{'C': 6.938525440729667, 'c_weight': None}

In [128]:
clf = LogisticRegression(C=6.94)
scores = []
for fold_labels_dict in folds_content_dicts[:-1]:
    clf.fit(fold_labels_dict["Train X scaled embeddings"], fold_labels_dict["Train y classes"])
    scores.append(clf.score(fold_labels_dict["Test X scaled embeddings"], fold_labels_dict["Test y classes"]))

np.mean(scores)

0.8681917211328976

In [129]:
test_set = folds_content_dicts[-1]
clf.fit(test_set["Train X scaled embeddings"], test_set["Train y classes"])
clf.score(test_set["Test X scaled embeddings"], test_set["Test y classes"])

0.8823529411764706

In [130]:
import pickle

print("Started writing data into a pickle file")
with open("data/cleaned_data/cross_val_split.pkl", "wb") as write_file:
    pickle.dump(folds_content_dicts, write_file) # encode dict into JSON
print("Done writing data into .pkl file")


Started writing data into a pickle file
Done writing data into .pkl file


In [5]:
with open("data/cleaned_data/cross_val_split.pkl", "rb") as read_file:
    test = pickle.load(read_file)

test[0]

{'Train y classes': [1,
  0,
  0,
  2,
  2,
  2,
  2,
  1,
  0,
  0,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  2,
  2,
  2,
  0,
  2,
  2,
  2,
  0,
  0,
  2,
  2,
  0,
  0,
  2,
  2,
  0,
  0,
  2,
  0,
  2,
  0,
  0,
  2,
  2,
  0,
  2,
  0,
  0,
  2,
  2,
  2,
  1,
  2,
  2,
  2,
  0,
  0,
  1,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  2,
  2,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  2,
  0,
  0,
  0,
  2,
  2,
  2,
  2,
  2,
  0,
  2,
  2,
  2,
  2,
  0,
  0,
  0,
  2,
  2,
  2,
  0,
  0,
  0,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  1,
  2,
  0,
  2,
  0,
  2,
  2,
  0,
  0,
  1,
  2,
  0,
  0,
  2,
  2,
  2,
  2,
  0,
  2,
  2,
  0,
  2,
  1,
  2,
  0,
  2,
  2,
  2,
  0,
  2,
  2,
  0,
  0,
  0,
  2,
  0,
  2,
  0],
 'Train y male share': [0.103,
  0.409,
  0.4,
  0.929,
  0.995,
  0.972,
  0.931,
  0.286,
  0.32,
  0.641,
  0.643,
  0.929,
  0.8,
  1.0,
  0.971,
  0.968,
  1.0,
  0.991,
  0.995,
 

In [115]:
test[0]["Train X"][0]

array([-0.04623144, -0.0083792 ,  0.01615657,  0.04223213, -0.03714952,
       -0.0181201 ,  0.06113513, -0.06018411,  0.06478234, -0.01866255,
       -0.02725041, -0.05047578, -0.0065047 ,  0.03774846, -0.09130936,
        0.12920116,  0.04534832,  0.0453199 , -0.02355766, -0.10057005,
        0.00541034, -0.01519444, -0.04499835,  0.04840711,  0.00131891,
       -0.05919844, -0.126063  ,  0.04036647, -0.026142  ,  0.01691947,
       -0.01142858, -0.08055522, -0.03978378, -0.0077859 , -0.00336136,
       -0.03115541,  0.03045835,  0.02449052,  0.04827584, -0.01721665,
        0.04537426, -0.01557749,  0.04987494,  0.01157437, -0.08826598,
       -0.10249634, -0.03807804,  0.00317594, -0.11364795,  0.04229956,
        0.0049036 , -0.02614813, -0.0474154 ,  0.00268679, -0.01969669,
        0.04532646, -0.04789288, -0.07651577, -0.0109855 , -0.0633727 ,
       -0.04095113,  0.01904656, -0.1029335 , -0.04801137,  0.01033369,
        0.01580972, -0.06155365,  0.08863799,  0.02082939,  0.01